<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_0_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [7]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    432      0 --:--:-- --:--:-- --:--:--   432
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [8]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [14]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [60]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [68]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.846536,4.217724,0.216805,03:49


In [0]:
# learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.657347,4.473830,0.262553,10:48


In [0]:
# learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.356186,4.441339,0.265599,10:50


In [0]:
# learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.317096,4.415151,0.267490,10:46


In [0]:
learn.save_encoder('ft_fold1_first')

In [70]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.129570,3.784860,0.368047,01:11


In [73]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.928550,3.145975,0.488757,01:24


In [74]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.438765,2.797564,0.543195,03:24


In [75]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.136801,2.488157,0.575148,03:48
1,1.983196,2.328965,0.600000,03:36


In [76]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.767577,2.166479,0.636686,03:34
1,1.578325,2.034510,0.668639,03:23


In [77]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.488597,1.893727,0.669823,03:35
1,1.348275,1.810190,0.685207,03:39


In [78]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.227859,1.768781,0.694675,03:21
1,1.124500,1.702821,0.719527,03:24


In [79]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.073736,1.706995,0.724260,03:20
1,0.982911,1.619249,0.728994,03:40


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.918071,1.583051,0.725444,03:23
1,0.939450,1.569021,0.744379,03:19
2,0.872300,1.488089,0.757396,03:38
3,0.772416,1.481102,0.757396,03:35


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.791307,1.504720,0.755030,03:29
1,0.810918,1.493133,0.756213,03:39
2,0.749855,1.428124,0.765680,03:33
3,0.669458,1.431251,0.766864,03:22


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.666615,1.462844,0.769231,03:48
1,0.702148,1.415295,0.766864,03:39
2,0.651262,1.471072,0.763314,03:43
3,0.591505,1.485025,0.766864,03:22


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.609190,1.511684,0.768047,03:37
1,0.640291,1.534306,0.758580,03:38
2,0.624592,1.469918,0.782248,03:44
3,0.521269,1.405253,0.777515,03:45


In [84]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.543814,1.524990,0.769231,03:44
1,0.599718,1.477387,0.770414,03:25
2,0.535624,1.494357,0.776331,03:36
3,0.518547,1.490324,0.776331,03:24


In [85]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.510630,1.546226,0.769231,03:36
1,0.569362,1.467090,0.771598,03:23
2,0.524490,1.511196,0.789349,03:34
3,0.484793,1.467686,0.788166,03:30


In [88]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.500035,1.542917,0.781065,03:36
1,0.553529,1.567656,0.788166,03:15
2,0.525578,1.495687,0.786982,03:38
3,0.478963,1.503427,0.789349,03:29


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-pubmed')

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [91]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
print(labels)
labels.size()

tensor([ 359,    2,  228,  334,    5,    8,    8,    8,    8,    8,    8,    8,
         685,  788,   21,  194,  610,   27,   27,  545,   29,   30,   30,   30,
          31,  584,   32,   34,   76,  289,  289,  289,  289,  289,  289,  289,
          38,   44,  183,   49,   49,   49,   49,   49,   49,   49,   49,   49,
          49,   49,   49,   49,   49,   49,   53,  113,   58,  427,   71,   71,
         491,   76,   76,  515,  289,  655,  655,  614,  214,  194,  526,  526,
         526,  526,  526,  526,  526,  526,  526,  526,  526,  526,  526,  526,
         526,  526,  526,  526,  526,  526,  526,  526,  526,  526,  526,  260,
         289,  289,  614,  614,   80,   80,   80,   80,   80,   80,   80,   80,
          80,   88,   89,   89,  268,   49,   91,   92,   94,   94,   94,   94,
          94,   94,   94,   94,   96,   96,   98,  683,   98,  100,  301,  100,
         100,  100,  100,  100,  100,  100,  100,  291,  236,  660,  660,  660,
         209,  512,  111,  111,  111, 10

torch.Size([867])